# Carga y tratado de datos

In [ ]:
import os
import os.path

import cudf
import cupy as cp
import cuml

import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN

%run ../utils/f_northing.py
%run ../utils/f_northing_numpy.py

#cities_to_use = ['sevilla']
#cities_to_use = ['shanghai']
cities_to_use = ['amsterdam', 'antwerp', 'asheville', 'austin', 'barcelona', 'barossa valley',
                 'barwon south west', 'beijing', 'belize', 'bergamo', 'sevilla', 'shanghai']

columns_to_use = ['id', 'host_id', 'host_response_rate', 'host_acceptance_rate',
                  'latitude', 'longitude', 'accommodates', 'price', 'number_of_reviews', 'reviews_per_month',
                 'neighbourhood_cleansed']

In [ ]:
%%time
listings = cudf.DataFrame()

for city in cities_to_use:
    directory = '../data/' + city + '/'
    if os.path.exists(directory):
        for file in os.listdir(directory):
            if file.endswith('.csv'):
                temp_df = cudf.read_csv(directory + file, usecols = columns_to_use)
                if 'price' in temp_df.columns:
                    if(temp_df['host_acceptance_rate'].dtype != 'object'):
                        temp_df['host_acceptance_rate'] = temp_df['host_acceptance_rate'].astype('object')
                    if(temp_df['neighbourhood_cleansed'].dtype != 'object'):
                        temp_df['neighbourhood_cleansed'] = temp_df['neighbourhood_cleansed'].astype('object')
                    if listings.size == 0:
                        listings = temp_df
                    else:
                        for column in listings.columns:
                            if listings[column].dtype != temp_df[column].dtype:
                                print('Found error: '+column+' type '+listings[column].dtype.name+' doesnt match '+temp_df[column].dtype.name)
                        listings = listings.append(temp_df)
                    
listings = listings.drop_duplicates().reset_index(drop=True)

listings['accommodates'] = listings['accommodates'].astype('int32')
listings['number_of_reviews'] = listings['number_of_reviews'].astype('int32')
listings['reviews_per_month'] = listings['reviews_per_month'].astype('float32').fillna(-1.0)
listings['neighbourhood_cleansed'], neighborhood_names = listings['neighbourhood_cleansed'].factorize()

listings['host_response_rate'] = listings['host_response_rate'].str.replace('%', '').fillna('-1').astype('int8')
listings['host_acceptance_rate'] = listings['host_acceptance_rate'].str.replace('%', '').fillna('-1').astype('int8')
listings['price'] = listings['price'].str.replace(['$', ','], '').astype('float32')

cupy_lat = cp.asarray(listings['latitude'])
cupy_long = cp.asarray(listings['longitude'])
n_cupy_array, e_cupy_array = latlong2osgbgrid_cupy(cupy_lat, cupy_long)
listings['northing'] = cudf.Series(n_cupy_array).astype('float32')
listings['easting'] = cudf.Series(e_cupy_array).astype('float32')

listings.head()

# Aplicación de un algoritmo k-means para visualizar clusters de listados

In [ ]:
%%time
dbscan = cuml.DBSCAN(eps=150)
price_df = listings[listings['price'] >= 200].reset_index(drop=True)
price_df['cluster'] = dbscan.fit_predict(price_df.loc[:, ['northing', 'easting']])
price_df['cluster'].nunique()

# Sección 2: K-means mediante CPU

## Carga y tratado de datos

In [ ]:
%%time
listings_cpu = pd.DataFrame()

for city in cities_to_use:
    directory = '../data/' + city + '/'
    if os.path.exists(directory):
        for file in os.listdir(directory):
            if file.endswith('.csv'):
                temp_df_cpu = pd.read_csv(directory + file, usecols = columns_to_use)
                if 'price' in temp_df_cpu.columns:
                    if(temp_df_cpu['host_acceptance_rate'].dtype != 'object'):
                        temp_df_cpu['host_acceptance_rate'] = temp_df_cpu['host_acceptance_rate'].astype('object')
                    if(temp_df_cpu['neighbourhood_cleansed'].dtype != 'object'):
                        temp_df_cpu['neighbourhood_cleansed'] = temp_df_cpu['neighbourhood_cleansed'].astype('object')
                    if listings_cpu.size == 0:
                        listings_cpu = temp_df_cpu
                    else:
                        for column in listings_cpu.columns:
                            if listings_cpu[column].dtype != temp_df_cpu[column].dtype:
                                print('Found error: '+column+' type '+listings_cpu[column].dtype.name+' doesnt match '+temp_df_cpu[column].dtype.name)
                        listings_cpu = listings_cpu.append(temp_df_cpu)
                    
listings_cpu = listings_cpu.drop_duplicates().reset_index(drop=True)

listings_cpu['accommodates'] = listings_cpu['accommodates'].astype('int32')
listings_cpu['number_of_reviews'] = listings_cpu['number_of_reviews'].astype('int32')
listings_cpu['reviews_per_month'] = listings_cpu['reviews_per_month'].astype('float32').fillna(-1.0)
listings_cpu['neighbourhood_cleansed'], neighborhood_names = listings_cpu['neighbourhood_cleansed'].factorize()

listings_cpu['host_response_rate'] = listings_cpu['host_response_rate'].fillna('-1').str.replace('%', '').astype('int32')
listings_cpu['host_acceptance_rate'] = listings_cpu['host_acceptance_rate'].fillna('-1').str.replace('%', '').astype('int32')
listings_cpu['price'] = listings_cpu['price'].fillna('-1').str.replace('$', '').str.replace(',', '').astype('float32')

numpy_lat = listings_cpu['latitude'].to_numpy()
numpy_long = listings_cpu['longitude'].to_numpy()
n_numpy_array, e_numpy_array = latlong2osgbgrid_numpy(numpy_lat, numpy_long)
listings_cpu['northing'] = pd.Series(n_numpy_array).astype('float32')
listings_cpu['easting'] = pd.Series(e_numpy_array).astype('float32')
listings_cpu.head()

# Aplicación de k-means

In [ ]:
%%time
dbscan_cpu = DBSCAN(eps=150)
price_df_cpu = listings_cpu[listings_cpu['price'] >= 200].reset_index(drop=True)
price_cpu_cluster = dbscan_cpu.fit_predict(price_df_cpu.loc[:, ('northing', 'easting')])
price_df_cpu['cluster'] = pd.array(price_cpu_cluster)
price_df_cpu['cluster'].nunique()

# Visualización de resultados mediante cuXfilter